#### ETL - Data Transformation

#### 1. Import libraries

In [29]:
import tweepy as tw
import re
import pandas as pd
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import re, string, unicodedata
import nltk
import datetime as dt
import emoji
from textblob import TextBlob

#### 2. We are interested in the sentiment of the following companies :

a) Netflix (username: netflix) \
b) Coca Cola (username: CocaCola) \
c) Tesla (username: Tesla)\
d) Nestle (username: Nestle) \
e) L'Oréal Group (username: Loreal) 

#### 3. Authorize Account for Twitter

In [3]:
consumer_key= ''
consumer_secret= ''
access_token= ""
access_token_secret= ''
callback_url = 'oob'

auth = tw.OAuthHandler(consumer_key, consumer_secret, callback_url)
redirect_url= auth.get_authorization_url()

print (redirect_url)

auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

user_pin_input = input ("What's the pin value? ")
api = tw.API (auth)

https://api.twitter.com/oauth/authorize?oauth_token=ofW6fgAAAAABNQviAAABeNIYIn8
What's the pin value? 2568788


#### 4. Get data from Twitter API 

In [72]:
def get_timeline(hashtag, company_id, document_id):
    
    number_of_tweets = 300
    tweets = []
    likes= []
    time= []
    user = []
    author = []
    retweet=[]
    
    # q = netflix hashtags 
    
    for i in tw.Cursor(api.search, q= hashtag, tweet_mode= 'extended', lang= "en").items(number_of_tweets):
        tweets.append(i.full_text)
        likes.append(i.favorite_count)
        time.append(i.created_at)
        retweet.append(i.retweet_count) 
        #location.append(i.user.location)
        
    df= pd.DataFrame({'tweets': tweets, 'likes': likes, 'timestamp': time, 'retweet': retweet})
    
    df["document_id"] = document_id
    df["company_id"] = company_id
    
    df= df[["company_id", "timestamp", "document_id", "tweets", "retweet", "likes"]]

    return df

In [73]:
get_timeline("netflix", 1, 1)

,company_id,timestamp,document_id,tweets,retweet,likes
0,1,2021-04-14 22:43:06,1,they took perks of being a wallflower off netf...,0,0
1,1,2021-04-14 22:43:05,1,RT @archivdob: LOVE AND MONSTERS CAME OUT TODA...,271,0
2,1,2021-04-14 22:43:05,1,RT @Crunchyroll: NEWS: Historically Inspired N...,220,0
3,1,2021-04-14 22:43:03,1,I already know “Glory” will be in at least one...,0,0
4,1,2021-04-14 22:43:03,1,RT @SeriesTWBZ: A Netflix anunciou 'Chadwick B...,299,0
...,...,...,...,...,...,...
295,1,2021-04-14 22:39:50,1,4 MINUTES INTO NETFLIX ND CHILL\n🤣🤣 https://t....,0,0
296,1,2021-04-14 22:39:50,1,@TPOImagination LOVE AND MONSTERS ON NETFLIX,0,0
297,1,2021-04-14 22:39:49,1,RT @OWENDZ: coming out means you get a netflix...,4,0
298,1,2021-04-14 22:39:49,1,@TPOImagination Yo quiero un meme de mica LOVE...,0,1


#### 5. Create a time dimension

In [60]:
def get_time_dim(df):
    df_time= df[["timestamp"]]
    df_time['Time'] = pd.to_datetime(df['timestamp']).dt.time
    df_time['Day']= pd.to_datetime(df['timestamp']).dt.day
    df_time['Month']= pd.to_datetime(df['timestamp']).dt.month
    df_time['Year']= pd.to_datetime(df['timestamp']).dt.year
    
    df_time.drop_duplicates(inplace = True)
    
    return df_time

## timestamp is the primary key here in SQL
## In fact table it is part of the primary key

#### 6. Create a sentiment score for fact table


In [25]:
def cleanup_tweets(s):
    text = s.lower().split()       # makes all characters lower case
    text = list(filter(lambda x: "http" not in x, text))        # removes https
    text = list(filter(lambda x: not x.startswith("@"), text))
    text = list(map(lambda x: emoji.demojize(x, delimiters=("", ",")).replace("_", " "), text))

    return " ".join(text)

print(cleanup_tweets("Game on 🎮\n\n📸 by @FromWhereICHG https://t.co/..."))

game on video game, camera with flash, by


In [26]:
df["cleaned_tweets"] = df["tweets"].apply(lambda x: cleanup_tweets(x))

df.head()

,tweets,likes,timestamp,user,retweet,cleaned_tweets
0,Netflix just dropped season 2 of The Circle 👀,0,2021-04-14 20:57:41,nomsed,0,"netflix just dropped season 2 of the circle eyes,"
1,RT @dianelyssa: Colton coming out and then ann...,0,2021-04-14 20:57:40,annabkrr,474,rt colton coming out and then announcing a net...
2,genshin taemin gay au kai taekook shinee tikto...,0,2021-04-14 20:57:40,filmhj,0,genshin taemin gay au kai taekook shinee tikto...
3,RT @netflix: Get your first look at Never Have...,0,2021-04-14 20:57:39,shoutoutlander,793,rt get your first look at never have i ever se...
4,@Austin49951405 @EDortGam3r @kittiwhores @Clue...,0,2021-04-14 20:57:39,wwbcomics,0,??? mineta is a pervert. what did they say wrong?


#### 7. Sentiment Analysis using Textblob

In [27]:
df['sentiment'] = df['cleaned_tweets'].apply(lambda tweet: TextBlob(tweet).sentiment)

In [28]:
df

,tweets,likes,timestamp,user,retweet,cleaned_tweets,sentiment
0,Netflix just dropped season 2 of The Circle 👀,0,2021-04-14 20:57:41,nomsed,0,"netflix just dropped season 2 of the circle eyes,","(0.0, 0.0)"
1,RT @dianelyssa: Colton coming out and then ann...,0,2021-04-14 20:57:40,annabkrr,474,rt colton coming out and then announcing a net...,"(0.35714285714285715, 0.5714285714285714)"
2,genshin taemin gay au kai taekook shinee tikto...,0,2021-04-14 20:57:40,filmhj,0,genshin taemin gay au kai taekook shinee tikto...,"(0.4166666666666667, 0.5833333333333334)"
3,RT @netflix: Get your first look at Never Have...,0,2021-04-14 20:57:39,shoutoutlander,793,rt get your first look at never have i ever se...,"(-0.0625, 0.41666666666666663)"
4,@Austin49951405 @EDortGam3r @kittiwhores @Clue...,0,2021-04-14 20:57:39,wwbcomics,0,??? mineta is a pervert. what did they say wrong?,"(-0.5, 0.9)"
...,...,...,...,...,...,...,...
295,RT @scratchAE: Why doesn’t Netflix let you tak...,0,2021-04-14 20:53:44,Batflash115,76,rt why doesn’t netflix let you take screenshot...,"(0.0, 0.0)"
296,RT @sirlovedrought: Hey @beyonce put some of y...,0,2021-04-14 20:53:43,BeyTHTBITCH,3,rt hey put some of your tours on netflix in th...,"(0.075, 0.275)"
297,"RT @JMatthiasFord: c'mon guys, who amongst us ...",0,2021-04-14 20:53:43,MiguelAMiggs,46,"rt c'mon guys, who amongst us hasn't stalked a...","(-0.5, 1.0)"
298,@JulllieyDark @CBS LOOK THIS @hulu @netflix @n...,1,2021-04-14 20:53:42,mello_tailine,1,look this #savemacgyver,"(0.0, 0.0)"


In [ ]:
#https://towardsdatascience.com/basic-tweet-preprocessing-in-python-efd8360d529e

#### Get data from Alpha Vantage

In [82]:
import requests 
import pandas as pd
import json
from pandas.io.json import json_normalize

key = "-"
URL= "https://www.alphavantage.co/query?function=OVERVIEW&symbol=AAPL&apikey=key"

In [85]:
def get_company_data():
    
    key = "-"
    URL= "-"
    r= requests.get(url= URL)
    data = r.json()
    df = json_normalize(data)   #transforms the data into a dataframe
    df  = df[["Name", "Sector", "Industry"]]   #returns only select columns
    
    return df

In [86]:
get_company_data()

<ipython-input-85-e8dc8ca52b31>:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(data)   #transforms the data into a dataframe


,Name,Sector,Industry
0,"Netflix, Inc",Communication Services,Entertainment


#### Get Data from News API - can also specify the publisher

In [87]:
import requests

In [98]:
#You can specify the date, source, and title mentions

def get_headlines():
    
    key = "-"
    URL= "http://newsapi.org/v2/everything?qInTitle=netflix&source=Forbes&language=en&from=2021-04-10&sortBy=publishedAt&apiKey=-"
    r= requests.get(url= URL)
    df = r.json()
    df = pd.DataFrame.from_dict(df)
    df = pd.concat([df.drop(['articles'], axis=1), df['articles'].apply(pd.Series)], axis=1)
    df [["Drop","source_"]] = df.source.apply(pd.Series)
    df = df [["title","publishedAt", "source_"]]

    return df

In [107]:
df_news = get_headlines()

In [108]:
df_news.head()

,title,publishedAt,source_
0,Stream It Or Skip It: ‘Dad Stop Embarrassing M...,2021-04-14T22:30:20Z,Decider
1,Free Netflix & Amazon Prime & Spotify & Airpor...,2021-04-14T22:20:55Z,Ozbargain.com.au
2,‘Shadow And Bone’ Creators Say The Netflix Ser...,2021-04-14T22:05:35Z,Forbes
3,"Stream It Or Skip It: ‘Law School’ On Netflix,...",2021-04-14T22:00:22Z,Decider
4,Stream It Or Skip It: ‘Why Did You Kill Me?’ o...,2021-04-14T21:40:41Z,Decider


In [105]:
df_news ["publishedAt"] = pd.to_datetime(s).dt.tz_localize(None)


In [106]:
df_news.head()

,title,publishedAt,source_
0,Stream It Or Skip It: ‘Dad Stop Embarrassing M...,2021-04-14 22:30:20+00:00,Decider
1,Free Netflix & Amazon Prime & Spotify & Airpor...,2021-04-14 22:20:55+00:00,Ozbargain.com.au
2,‘Shadow And Bone’ Creators Say The Netflix Ser...,2021-04-14 22:05:35+00:00,Forbes
3,"Stream It Or Skip It: ‘Law School’ On Netflix,...",2021-04-14 22:00:22+00:00,Decider
4,Stream It Or Skip It: ‘Why Did You Kill Me?’ o...,2021-04-14 21:40:41+00:00,Decider


In [ ]:
#newspapers= Daily Mail, Extra, Variety, 